<img src="https://avatars.githubusercontent.com/u/74911464?s=200&v=4"
     alt="OpenEO Platform logo"
     style="float: left; margin-right: 10px;" />
# OpenEO Platform - Client Side Processing

## Data Fusion


In this interactive notebook we will show some usage eaxample of the Client Side Processing functionality added recently to the openEO Python client.

## Requirements

To use this functionality, you need `3.9<=Python<3.11`.

You can install it using:
    `pip install openeo[localprocessing]`

<div class="alert alert-block alert-warning">
This functionality is still under development and the installation procedure might change.
Please refer to official documentation page for the most up to date instructions:
    
<a href="url">https://open-eo.github.io/openeo-python-client/cookbook/localprocessing.html</a>

## Sample Datasets

Clone the repository containing sample datasets provided by openEO Platform:

In [ ]:
import os
if not os.path.exists('./openeo-localprocessing-data'):
    !git clone https://github.com/Open-EO/openeo-localprocessing-data.git

Initialize the local collections and inspect them:

In [ ]:
from openeo.local import LocalConnection
local_conn = LocalConnection(['./openeo-localprocessing-data'])
local_conn.list_collections()

**We are going to work with the sample Sentinel-2 and Sentinel-1 data.**

Let's look at them step by step:

In [ ]:
s2 = local_conn.load_collection('openeo-localprocessing-data/sample_netcdf/S2_L2A_sample.nc')
s2_dataarray = s2.execute()
s2_dataarray

In [ ]:
s1 = local_conn.load_collection('openeo-localprocessing-data/sample_netcdf/S1_GRD_sample.nc')
s1_dataarray = s1.execute()
s1_dataarray

We notice that the Sentinel-1 data covers a bigger area than the one covered by Sentinel-2. This is also visible in the local collections metadata.

In this case the two datacubes have the same resolution and are probably also aligned. However, sometimes we might need to combine data coming from different satellite sources with different projections and resolution.

In this scenario we could use the `resample_cube_spatial` openEO process that allows to reproject and crop the source to match a target datacube in one shot.

Have a look at the official process description if you want to know more details: https://processes.openeo.org/#resample_cube_spatial

We now want to resample the S1 data to match S2, for being able to merge them later:

In [ ]:
s1_resampled = s1.resample_cube_spatial(target=s2,method="bilinear")
s1_resampled.execute()

Now our Sentinel-1 data has the same spatial extent and resolution. However, the two datacubes have different number of time steps and we have to align that dimension as well to make a meaningful merge.

We could compute a temporal aggregation using the openEO process `aggregate_temporal_period`, to compute the monthly median values.
Have a look at the official process description if you want to know more details: https://processes.openeo.org/#aggregate_temporal_period

In [ ]:
s2_monthly_median = s2.aggregate_temporal_period(period="month",reducer="median")
s1_monthly_median = s1_resampled.aggregate_temporal_period(period="month",reducer="median")

Now that spatial and temporal dimensions are aligned, we can proceed merging the two datacubes and look at the generated openEO process graph:

In [ ]:
s2_s1_monthly_median = s2_monthly_median.merge_cubes(s1_monthly_median)
s2_s1_monthly_median

Finally execute the process and check the output:

In [ ]:
s2_s1_monthly_median = s2_s1_monthly_median.execute()
s2_s1_monthly_median

The output now contains the optical bands of Sentinel-2 and the two polarizations from Sentinel-1.

The datacube is aligned spatially and temporally, which was possible using the `resample_cube_spatial` and `aggregate_temporal_period` processes.